In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as mlt
import seaborn as sp
from torch.autograd import Variable
from torch import autograd
from datetime import datetime
import matplotlib.pyplot as plt
import argparse
from datetime import timedelta
import torch.autograd.functional as F

In [4]:
def gen_batch_forecasting(batch_size, step, dset):
    real_dset = dset[step * batch_size: (step + 1) * batch_size]
    return real_dset

In [2]:
def train_mogrifier_lstm(model, optimizer_model, loss_function, real_train, real_test, step_per_epoch):
    
    errors_generator = []
    real_dataset = torch.empty((0, future_step, input_size), dtype=torch.float32).to(device)
    gen_dataset = torch.empty((0, future_step, input_size), dtype=torch.float32).to(device)
    epoch = 100
    
    for i in range(epoch):
        count = 0
        for step in range(step_per_epoch):
            # Data for training the discriminator
            real_data = gen_batch_forecasting(batch_size, count, real_train)
            real_label = gen_batch_forecasting(batch_size, count, real_test)

            model.zero_grad()
            generated_samples = model(real_data)

            L2 = torch.sum((real_label - generated_samples)**2)
            g_loss = loss_function(generated_samples, real_label) + L2
            g_loss.backward()
            optimizer_model.step()
            errors_generator.append(g_loss.item())
            
            count += 1
            if i == epoch - 1:
                real_dataset = torch.cat([real_dataset, real_label], dim=0)
                gen_dataset = torch.cat([gen_dataset, generated_samples], dim=0)

            # Show loss
            if step == step_per_epoch - 1:
                print(f"Epoch: {i} Loss G.: {g_loss.item()}")

    return real_dataset, gen_dataset, errors_generator

In [34]:
def train_bi_lstm(model, optimizer_model, loss_function, real_train, real_test, step_per_epoch):
    
    errors_generator = []
    real_dataset = torch.empty((0, 8), dtype=torch.float32).to(device)
    gen_dataset = torch.empty((0, 8), dtype=torch.float32).to(device)
    epoch = 400
    
    for i in range(epoch):
        count = 0
        for step in range(step_per_epoch):
            # Data for training the discriminator
            real_data = gen_batch_forecasting(batch_size, count, real_train)
            real_label = gen_batch_forecasting(batch_size, count, real_test)

            model.zero_grad()
            generated_samples = model(real_data)

            L2 = torch.sum((real_label - generated_samples)**2)
            g_loss = loss_function(generated_samples, real_label) + L2
            g_loss.backward()
            optimizer_model.step()
            errors_generator.append(g_loss.item())
            
            count += 1
            if i == epoch - 1:
                real_dataset = torch.cat([real_dataset, real_label], dim=0)
                gen_dataset = torch.cat([gen_dataset, generated_samples], dim=0)

            # Show loss
            if step == step_per_epoch - 1:
                print(f"Epoch: {i} Loss G.: {g_loss.item()}")

    return real_dataset, gen_dataset, errors_generator

In [35]:
def train_vanilla_lstm(model, optimizer_model, loss_function, real_train, real_test, step_per_epoch):
    
    errors_generator = []
    real_dataset = torch.empty((0, 8), dtype=torch.float32).to(device)
    gen_dataset = torch.empty((0, 8), dtype=torch.float32).to(device)
    epoch = 400
    
    for i in range(epoch):
        count = 0
        for step in range(step_per_epoch):
            # Data for training the discriminator
            real_data = gen_batch_forecasting(batch_size, count, real_train)
            real_label = gen_batch_forecasting(batch_size, count, real_test)

            model.zero_grad()
            generated_samples = model(real_data)

            L2 = torch.sum((real_label - generated_samples)**2)
            g_loss = loss_function(generated_samples, real_label) + L2
            g_loss.backward()
            optimizer_model.step()
            errors_generator.append(g_loss.item())
            
            count += 1
            if i == epoch - 1:
                real_dataset = torch.cat([real_dataset, real_label], dim=0)
                gen_dataset = torch.cat([gen_dataset, generated_samples], dim=0)

            # Show loss
            if step == step_per_epoch - 1:
                print(f"Epoch: {i} Loss G.: {g_loss.item()}")

    return real_dataset, gen_dataset, errors_generator

In [56]:
def train_scinet(model, loss_function, optim, real_train, real_test, step_per_epoch):
        epoch_start = 0
        loss_evol = []
        epochs = 30
        real_dataset = torch.empty((0, future_step, input_size), dtype=torch.float32).to(device)
        gen_dataset = torch.empty((0, future_step, input_size), dtype=torch.float32).to(device)
        
        for epoch in range(epochs):
            model.train()
            total_loss = 0.0
            epoch_loss = 0.0
            count = 0
            #adjust_learning_rate(optim, epoch, self.args)

            for step in range(step_per_epoch):

                inputs = gen_batch_forecasting(batch_size, count, real_train)
                targets = gen_batch_forecasting(batch_size, count, real_test)

                # inference
                model.zero_grad()
                if stacks == 1:
                    forecast = model(inputs)
                    loss = loss_function(forecast, targets)
                if stacks == 2:
                    forecast, mid = model(inputs)
                    loss = loss_function(forecast, targets) + loss_function(mid, targets)
                epoch_loss += loss.item()

                # backpropagate
                loss.backward()
                total_loss += loss.item()
                optim.step()

                count += 1
                if epoch == epochs - 1:
                    real_dataset = torch.cat([real_dataset, targets], dim=0)
                    gen_dataset = torch.cat([gen_dataset, forecast], dim=0)

            epoch_loss /= step_per_epoch # average loss per batch
            loss_evol.append(epoch_loss)
            print(f"Epoch: {epoch} Loss G.: {epoch_loss}")

        return real_dataset, gen_dataset, loss_evol